Tokenization is the task of segmenting a string into *tokens*. A token often corresponds to a word (in the context of computational linguistics), though it may correspond to other kinds of strings, such as punctuation. 

Formally, we say that a tokenizer maps a string (in $\Sigma^*$) into a string of strings (in $\Sigma^{**}$).  

$$\text{tokenize}: \Sigma^* \rightarrow \Sigma^{**}$$

We can generally do tokenization reasonably well in English by finding substrings that have white space on either side. We could do this in Python using `str.split()`, though a regular expression will turn out to be more useful.

In [1]:
import re

news = ("Lin Xingzhi said that the trade war prompted the relocation of "
        "factories in China to Vietnam, Laos, India, Taiwan and even Malaysia.")

re.findall("(?:(.*?)(?:\s|$))+?", news)

['Lin',
 'Xingzhi',
 'said',
 'that',
 'the',
 'trade',
 'war',
 'prompted',
 'the',
 'relocation',
 'of',
 'factories',
 'in',
 'China',
 'to',
 'Vietnam,',
 'Laos,',
 'India,',
 'Taiwan',
 'and',
 'even',
 'Malaysia.',
 '']

Note that this implementaton of $\text{tokenize}$ is furthermore invertible:

$$\text{tokenize}^{-1}: \Sigma^{**} \rightarrow \Sigma^*$$

That is, can recover the original string from the tokenized variant, in this case by simply joining all tokens by whitespace. **Not all implementations of $\text{tokenize}$, viewed as functions from strings to strings of strings, are invertible**. 

For instance, note that, even for a short piece of text like this, we're missing certain things we might want to capture:

1. In the case of the name `Lin Xingzhi`, we are probably oversegmenting. Really, we probably want to be a single token. We'll deal with this when we introduce the task of *named entity recognition* (NER).
2. In the case of the list of country names, some country names have punctuation on the end (`Vietnam,`, `Laos,`, `India,`, `Malaysia.`), while others don't (`Taiwan`).

To handle these, we might try to introduce additional tokenization rules by augmenting our regular expression.

In [2]:
# this regular expression won't work for general comma-separated lists
re.findall("(?:([^A-Z].*?|(?:[A-Z].*?(?:\s?))+)(?:,?\s|\.$))+?", news)

['Lin Xingzhi',
 'said',
 'that',
 'the',
 'trade',
 'war',
 'prompted',
 'the',
 'relocation',
 'of',
 'factories',
 'in',
 'China',
 'to',
 'Vietnam',
 'Laos',
 'India',
 'Taiwan',
 'and',
 'even',
 'Malaysia']

This works reasonably well in this case, but we are actually throwing out punctuation, where generally a tokenizer will retain it as its own token. This fact implies that we cannot reconstruct the original string from it's tokenization. And even if we were to augment our tokenizer to retain the punctuation as separate tokens...

In [3]:
# this regular expression won't work for general comma-separated lists
re.findall("(?:((?<=\s)[^A-Z,\.][^,\.]*?|(?:[A-Z][^,\.]*?(?:\s?))+|[,\.])(?=,?\s|\.$))+?", news)

['Lin Xingzhi',
 'said',
 'that',
 'the',
 'trade',
 'war',
 'prompted',
 'the',
 'relocation',
 'of',
 'factories',
 'in',
 'China',
 'to',
 'Vietnam',
 ',',
 'Laos',
 ',',
 'India',
 ',',
 'Taiwan',
 'and',
 'even',
 'Malaysia']

...we wouldn't be able to unambiguously reconstruct the string, because we couldn't be sure whether a whitespace should come before or after a punctuation mark. (We could devise heuristics based on our knowledge of English orthographic conventions to get close, but we couldn't know for sure.)  

Thus, if we want to be able to reconstruct the original string (and we don't necessarily always do), we need to have $\text{tokenize}$ produce additional information in order to reconstruct the original string: usually some form of *character offset*, which tells us the beginning and ending points of a token relative to the beginning of the original string.

$$\text{tokenization-offset}: \Sigma^* \rightarrow (\mathbb{N} \times \mathbb{N})^{*}$$

We could then use $\text{tokenization-offset}$ to produce a tokenization by indexing into the original string. (These offsets either need to cover the entire input string, or we need to know what characters to default to for any spans of the original string, such as white-space characters, we didn't include in the tokenization.)

Ignoring the reconstruction issue, though, we will see that the regular expression above won't work in general. For instance, our current regular expression doesn't handle contractions (`'ll`, `'d`, `n't`, etc.), which we generally want to treat as separate tokens (since they are semantically equivalent to full words: `will`, `would`, `not`, etc.), and it won't handle other kinds of punctuation (e.g. `-`), which could be word-internal or its own token. It also doesn't handle names (and more generally, multiword expressions) that don't fit a very specific format involving title-casing.

These issues get much more difficult when we move outside of English. For instance, consider the following Chinese text from [this article](https://www.post852.com/270084/%E4%B8%AD%E7%92%B0/).

Example from [this post](https://medium.com/@jjsham/nlp-tokenizing-chinese-phases-3302da4336bf).

In [4]:
news = "林行止表示貿易戰促使在中國的工廠搬遷到越南、寮國、印度、台灣甚至是馬來西亞"

Google Translate translates the article as follows:

`Lin Xingzhi said that the trade war prompted the relocation of factories in China to Vietnam, Laos, India, Taiwan and even Malaysia.`

If you didn't know anything about Chinese, the lack of white space here might make you think that it is highly synthetic; but it is the opposite: Chinese is highly analytic. Either way, even the a really fancy regular expression won't work here for.

We could of course try splitting every character apart.

In [5]:
list(news)

['林',
 '行',
 '止',
 '表',
 '示',
 '貿',
 '易',
 '戰',
 '促',
 '使',
 '在',
 '中',
 '國',
 '的',
 '工',
 '廠',
 '搬',
 '遷',
 '到',
 '越',
 '南',
 '、',
 '寮',
 '國',
 '、',
 '印',
 '度',
 '、',
 '台',
 '灣',
 '甚',
 '至',
 '是',
 '馬',
 '來',
 '西',
 '亞']

This doesn't work for reasons similar to English. Just like in English, words in Chinese can be multiple characters long. For instance, `林行止` (Lin Xingzhi) and `馬來西亞` are names (Malaysia). But even worse, most characters can stand on there own as words as well. For instance, `林行止` separated by character would be the nonsense phrase "forest walk halt" and `馬來西亞` separated by character would be "horse come to Western Asia". 

The more general point is that good tokenization requires knowing the context of a token. This is true even in English: consider the string `New York`. In the context of the sentence `I live in Rochester, New York`, we probably want `New York` to be a token. But in the context of `I read the New York Times this morning`, we probably don't: we want `New York Times` instead.

Again, we might be able to devise heuristics for handling this case–like take the maximal string, but it's still not going to handle multiword expressions like `run the gamut`, where we're generally not going to have orthographic conventions like title-casing to help us out: should we tokenize this as `run the gamut` or `run`, `the`, `gamut`?

So what should we do? For the purposes of this course, our answer is going to be "use existing packages"–largely because actually solving the tokenization problem, even in English, turns out to be really hard; and our best systems are ones that require fairly heavy machine learning-based approaches (covered in LING281/481 and LING282/482). 

I'll discuss two of the most popular state-of-the-art packages.

- [`stanza`](https://stanfordnlp.github.io/stanza/)
- [`spacy`](https://spacy.io/)

You may have also run across the [`nltk`](https://www.nltk.org/). I strongly advise aginst using `nltk` for any of the text normalization we're going to cover today. The packages I'll discuss are so much better that it's not even worth considering. (`nltk` can still be useful for loading particular kinds of resources like [`wordnet`](https://wordnet.princeton.edu/), and we will use it in Assignmnt 6.)

## Stanza

To use `stanza` for a particular language, we then need to download the relevant language-specific models.

In [7]:
import stanza

# Download an English model
stanza.download('en')
  
# Similarly, download a (traditional) Chinese model
stanza.download('zh')

INFO:stanza:Downloading default packages for language: en (English) ...


INFO:stanza:Finished downloading models and saved to /root/stanza_resources.


INFO:stanza:"zh" is an alias for "zh-hans"
INFO:stanza:Downloading default packages for language: zh-hans (Simplified_Chinese) ...


INFO:stanza:Finished downloading models and saved to /root/stanza_resources.


(Source: https://bit.ly/2kb8eJU)

We process text using `stanza.Pipeline`s, which contain different `Processor` units. The pipeline is language-specific, so again you'll need to first specify the language. By default, `Pipeline`s run a bunch of processors, including ones we're not going to be looking at today. For now, we'll specify that we just want the `tokenize` processor.^[We also need to specify whether we want to use a GPU, since the models in `stanza` are deep learning models and can benefit from them. We won't use them hre by setting `use_gpu=False`.]

In [8]:
# Build an English pipeline, with all processors by default
stanza_en_nlp = stanza.Pipeline(lang='en', processors = 'tokenize', use_gpu=False)

# Build a Chinese pipeline, with customized processor list, and force it to use CPU
stanza_zh_nlp = stanza.Pipeline(lang='zh', processors='tokenize', use_gpu=False)

### A full processors list looks like: processors='tokenize,lemma,pos,depparse'

INFO:stanza:Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES


INFO:stanza:Loading these models for language: en (English):
| Processor | Package  |
------------------------
| tokenize  | combined |

INFO:stanza:Use device: cpu
INFO:stanza:Loading: tokenize
INFO:stanza:Done loading processors!
INFO:stanza:Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES


INFO:stanza:"zh" is an alias for "zh-hans"
INFO:stanza:Loading these models for language: zh-hans (Simplified_Chinese):
| Processor | Package |
-----------------------
| tokenize  | gsdsimp |

INFO:stanza:Use device: cpu
INFO:stanza:Loading: tokenize
INFO:stanza:Done loading processors!


#### Creating document objects

In [9]:
en_str = "Homelessness is to be found in the large numbers of mentally ill and substance-abusing people in the homeless population, don't you think?"
zh_str = "林行止表示貿易戰促使在中國的工廠搬遷到越南、寮國、印度、台灣甚至是馬來西亞"

In [10]:
## Our earlier algo using regex function
import re
print(re.findall('[^ ]+', en_str))  ## based on whitespace seperation

['Homelessness', 'is', 'to', 'be', 'found', 'in', 'the', 'large', 'numbers', 'of', 'mentally', 'ill', 'and', 'substance-abusing', 'people', 'in', 'the', 'homeless', 'population,', "don't", 'you', 'think?']


In [11]:
# Processing English text
en_doc = stanza_en_nlp(en_str)
print(type(en_doc))

# Processing Chinese text
zh_doc = stanza_zh_nlp(zh_str)
print(type(zh_doc))

<class 'stanza.models.common.doc.Document'>
<class 'stanza.models.common.doc.Document'>


In [12]:
for sent in en_doc.sentences:
    stanza_en_sent = [word.text for word in sent.words]
    print(stanza_en_sent)

['Homelessness', 'is', 'to', 'be', 'found', 'in', 'the', 'large', 'numbers', 'of', 'mentally', 'ill', 'and', 'substance', '-', 'abusing', 'people', 'in', 'the', 'homeless', 'population', ',', 'do', "n't", 'you', 'think', '?']


In [13]:
for i, sent in enumerate(zh_doc.sentences):
    stanza_zh_sent = [word.text for word in sent.words]
    print(stanza_zh_sent)

['林', '行止', '表示', '貿易', '戰', '促使', '在', '中國', '的', '工廠', '搬遷', '到', '越南', '、', '寮國', '、', '印度', '、', '台灣', '甚至是', '馬', '來西', '亞']


## Spacy

As with `stanza`, when using `spacy`, we need to download language-specific models.

In [15]:
import spacy

!python -m spacy download en_core_web_sm
!python -m spacy download zh_core_web_sm
!python -m spacy download it_core_news_sm

/usr/local/lib/python3.8/dist-packages/torch/cuda/__init__.py:497: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")


/usr/local/lib/python3.8/dist-packages/torch/cuda/__init__.py:497: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")
2023-02-06 17:17:12.352179: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 40.8 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
/usr/local/lib/python3.8/dist-packages/torch/cuda/__init__.py:497: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")
2023-02-06 17:17:29.485403: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━

 We then need to load the models.

In [16]:
spacy_en_nlp = spacy.load("en_core_web_sm") #disable=["tagger", "parser", "ner"]
spacy_zh_nlp = spacy.load("zh_core_web_sm") #disable=["tagger", "parser", "ner"]

#### Creating document objects

In [17]:
en_doc = spacy_en_nlp(en_str)
spacy_en_sent = [w.text for w in en_doc]
print(spacy_en_sent)

['Homelessness', 'is', 'to', 'be', 'found', 'in', 'the', 'large', 'numbers', 'of', 'mentally', 'ill', 'and', 'substance', '-', 'abusing', 'people', 'in', 'the', 'homeless', 'population', ',', 'do', "n't", 'you', 'think', '?']


In [18]:
zh_doc = spacy_en_nlp(zh_str)
spacy_zh_sent = [w.text for w in zh_doc]
print(spacy_zh_sent)

['林行止表示貿易戰促使在中國的工廠搬遷到越南、寮國、印度、台灣甚至是馬來西亞']
